In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import os
import cv2
import numpy as np
import pandas as pd

C:\Users\pajag\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pajag\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4sp5sua7cbgxueoc35yp2asoicyyeqzz.gfortran-win_amd64.dll
C:\Users\pajag\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
from sklearn.model_selection import train_test_split

In [3]:
def load_images_from_folder(folder_path):
    
    image_filenames = os.listdir(folder_path)
    images = []
    i = 0
    for image_filename in image_filenames:
        image = cv2.imread(os.path.join(folder_path, image_filename))
        images.append(image)
        i = i+1
    return np.array(images)

In [4]:
real_test = load_images_from_folder('C://Users//pajag//OneDrive//Desktop//frame//test_real')

In [5]:
fake_test = load_images_from_folder('C://Users//pajag//OneDrive//Desktop//frame//test_fake')

In [6]:
real = load_images_from_folder('C://Users//pajag//OneDrive//Desktop//frame//real')

In [7]:
fake = load_images_from_folder('C://Users//pajag//OneDrive//Desktop//frame//fake')

In [8]:
real.shape

(1504, 224, 224, 3)

In [9]:
fake.shape

(1280, 224, 224, 3)

In [10]:
real.shape

(1504, 224, 224, 3)

In [11]:
def preprocess_images(array):
    resized = []
    for img in array:
        img = img / 255.0
        resized.append(img)
    return resized

In [12]:
real_images = preprocess_images(real)

In [13]:
fake_images = preprocess_images(fake)

In [14]:
real_test_pre = preprocess_images(real_test)

In [15]:
fake_test_pre = preprocess_images(fake_test)

In [16]:
import pandas as pd

In [17]:
real_df = pd.DataFrame({'image':real_images, 'label': '0'})

In [18]:
fake_df = pd.DataFrame({'image':fake_images, 'label': '1'})

In [19]:
real_test_df = pd.DataFrame({'image':real_test_pre, 'label': '0'})

In [20]:
fake_test_df = pd.DataFrame({'image':fake_test_pre, 'label': '1'})

In [21]:
df = pd.concat([real_df,fake_df])

In [22]:
len(df)

2784

In [23]:
df = df.iloc[np.random.permutation(len(df))]

In [24]:
df_test = pd.concat([real_test_df,fake_test_df])

In [25]:
len(df_test)

1057

In [26]:
df_test = df.iloc[np.random.permutation(len(df_test))]

In [27]:
x_batch = df['image']
y_batch = df['label']

In [28]:
x_batch = [tf.convert_to_tensor(img) for img in x_batch]
x_batch = tf.stack(x_batch)
y_batch = tf.convert_to_tensor(y_batch)
y_batch = tf.strings.to_number(y_batch, out_type=tf.float32)

In [29]:
x_batch_test = df_test['image']
y_batch_test = df_test['label']

In [30]:
x_batch_test = [tf.convert_to_tensor(img) for img in x_batch_test]
x_batch_test = tf.stack(x_batch_test)
y_batch_test = tf.convert_to_tensor(y_batch_test)
y_batch_test = tf.strings.to_number(y_batch_test, out_type=tf.float32)

In [31]:
IMAGE_SIZE = (224, 224)

In [32]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [33]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [34]:
model = model.fit(x_batch,y_batch,batch_size=5,epochs=5,validation_split=0.2)

Epoch 1/5
446/446 [==============================] - 199s 438ms/step - loss: 0.6213 - accuracy: 0.6444 - val_loss: 0.5361 - val_accuracy: 0.7307
Epoch 2/5
446/446 [==============================] - 197s 443ms/step - loss: 0.4895 - accuracy: 0.7652 - val_loss: 0.4632 - val_accuracy: 0.7648
Epoch 3/5
446/446 [==============================] - 191s 428ms/step - loss: 0.4569 - accuracy: 0.7813 - val_loss: 0.4222 - val_accuracy: 0.7953
Epoch 4/5
446/446 [==============================] - 193s 432ms/step - loss: 0.3855 - accuracy: 0.8092 - val_loss: 0.3579 - val_accuracy: 0.8115
Epoch 5/5
446/446 [==============================] - 163s 366ms/step - loss: 0.3407 - accuracy: 0.8433 - val_loss: 0.3895 - val_accuracy: 0.8205


In [35]:
loss = model.history['loss']

In [36]:
loss

[0.6212847828865051,
 0.4894959330558777,
 0.4569321274757385,
 0.385530561208725,
 0.3407454788684845]

In [ ]:
def save_model(model, filename):
    model.save(filename)

In [ ]:
def evaluate_model(data):
    test_images = [tf.convert_to_tensor(image) for image in data['image'].values]
    test_images = tf.stack(test_images)

    test_labels = tf.convert_to_tensor(data['label'].values)
    test_labels = tf.strings.to_number(test_labels, out_type=tf.float32)

    loss, accuracy = model.evaluate(test_images, test_labels)
    return loss,accuracy

In [ ]:
loss,accurcy = evaluate_model(df_test)


In [ ]:
save_model(model, "deepfake_detection_vid_model.h5")